In [2]:
from google.colab import drive
from pathlib import Path
import os

drive.mount("/content/drive")

PROJECT = Path("/content/drive/MyDrive/gb-flex-desk")
PROJECT.mkdir(parents=True, exist_ok=True)
os.chdir(PROJECT)

Path("data/raw").mkdir(parents=True, exist_ok=True)
Path("data/processed").mkdir(parents=True, exist_ok=True)

print("You are now in:", Path.cwd())
print("This should be True if file exists:", Path("data/processed/prices.parquet").exists())


Mounted at /content/drive
You are now in: /content/drive/MyDrive/gb-flex-desk
This should be True if file exists: True


In [3]:
import requests
import pandas as pd
from pathlib import Path

BASE = "https://data.elexon.co.uk/bmrs/api/v1"

def system_prices_day(settlement_date: str) -> pd.DataFrame:
    url = f"{BASE}/balancing/settlement/system-prices/{settlement_date}"
    r = requests.get(url, params={"format":"json"}, timeout=30)
    r.raise_for_status()
    return pd.DataFrame(r.json()["data"])

def add_sp_timestamp(df: pd.DataFrame) -> pd.DataFrame:
    d = pd.to_datetime(df["settlementDate"])
    sp = df["settlementPeriod"].astype(int)
    out = df.copy()
    out["ts"] = d + pd.to_timedelta((sp - 1) * 30, unit="min")
    return out.sort_values("ts")

raw = Path("data/raw")
processed = Path("data/processed")

def get_day(date_str: str) -> pd.DataFrame:
    f = raw / f"system_prices_{date_str}.parquet"
    if f.exists():
        return pd.read_parquet(f)
    df = add_sp_timestamp(system_prices_day(date_str))
    df.to_parquet(f, index=False)
    return df

# Build last 365 days instead of 2 weeks
end = pd.Timestamp.today().normalize() - pd.Timedelta(days=1)  # yesterday
start = end - pd.Timedelta(days=365)
dates = pd.date_range(start, end, freq="D")
print("Pulling", len(dates), "days from", dates.min().date(), "to", dates.max().date())

# Pull each day (uses cache if already downloaded). Skip empty days to avoid warnings.
days = []
for d in dates:
    day = get_day(d.strftime("%Y-%m-%d"))
    if len(day) > 0:
        days.append(day)

all_days = pd.concat(days, ignore_index=True)

prices = all_days[["ts", "systemSellPrice", "systemBuyPrice"]].rename(
    columns={"systemSellPrice": "ssp", "systemBuyPrice": "sbp"}
).sort_values("ts")

outpath = processed / "prices.parquet"
prices.to_parquet(outpath, index=False)

print("Saved to:", outpath.resolve())
print("Rows:", len(prices))
prices.head()

Pulling 366 days from 2024-12-06 to 2025-12-06


/tmp/ipython-input-1626947483.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_days = pd.concat(days, ignore_index=True)


Saved to: /content/drive/MyDrive/gb-flex-desk/data/processed/prices.parquet
Rows: 17568


,ts,ssp,sbp
0,2024-12-06 00:00:00,2.98,2.98
1,2024-12-06 00:30:00,7.24,7.24
2,2024-12-06 01:00:00,1.56,1.56
3,2024-12-06 01:30:00,56.19,56.19
4,2024-12-06 02:00:00,22.60,22.60


from matplotlib import pyplot as plt
_df_0['ssp'].plot(kind='hist', bins=20, title='ssp')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['ts']
  ys = series['sbp']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_4.sort_values('ts', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('ts')
_ = plt.ylabel('sbp')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['ts']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'ts'}, axis=1)
              .sort_values('ts', ascending=True))
  xs = counted['ts']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_5.sort_values('ts', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('ts')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['ts']
  ys = series['ssp']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_6.sort_values('ts', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('ts')
_ = plt.ylabel('ssp')

from matplotlib import pyplot as plt
_df_7['ssp'].plot(kind='line', figsize=(8, 4), title='ssp')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_8['sbp'].plot(kind='line', figsize=(8, 4), title='sbp')
plt.gca().spines[['top', 'right']].set_visible(False)